In [3]:
# 3.1 딥러닝의 원리
# 퍼셉트론 : 뉴런과 같은 기본역할

#3.1.2 심층 신경망

from sklearn import datasets
import torch
import torch.nn as nn # 신경망을 구현하는데 필요한 기능 구현
import torch.optim as optim #옵티마이저 구현

dataset = datasets.load_boston()
X, y = dataset['data'], dataset['target']

X = torch.FloatTensor(X)
y = torch.FloatTensor(y).unsqueeze(-1)

X = (X - torch.mean(X) / torch.std(X)) #데이터 범위 조정을 위해 표준화 적용
model = nn.Linear(13, 1)

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 손실함수로 모델이 추론한 타깃과 실제 타깃 사이의 차이, 오차를 계산
# 미분으로 손실 함수의 기울기를 계산, 가중치를 어떻게 수정 오차가 줄어드는지 파악
# => '에포크(epoch)'

def train(model, criterion, optimizer, X, y):
    optimizer.zero_grad() # 기울기 초기화
    hypothesis = model(X) # 모델을 사용해 타깃을 추론
    loss = criterion(hypothesis, y) # 오차를 계산
    loss.backward() # 기울기 계산
    optimizer.step() # 경사 하강법으로 가중치를 수정
    return loss.item() #에포크 오차 반환

n_epochs = 100
for epoch in range(1, n_epochs+1):
    loss = train(model, criterion, optimizer, X, y)

    if epoch % 10 == 0:
        print(epoch, loss)



10 inf
20 nan
30 nan
40 nan
50 nan
60 nan
70 nan
80 nan
90 nan
100 nan


In [11]:
# 로지스틱 회귀 : 선형회귀에 시그모이드 함수를 추가 , 이진 크로스 엔트로피 손실 함수를 사용

from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim


dataset = datasets.load_breast_cancer()

X, y = dataset['data'], dataset['target']

X = torch.FloatTensor(X)
y = torch.FloatTensor(y).view(-1, 1)

X = (X - torch.mean(X) / torch.std(X)) # 표준화 적용

model = nn.Sequential(nn.Linear(30, 1),nn.Sigmoid())
criterion = nn.BCELoss() # 이진 크로스 엔트로피 손실 함수 객체를 생성

optimizer = optim.SGD(model.parameters(), lr=0.1) # 확율적 경사 하강법 옵티마이저 객체

n_epochs = 100
for epoch in range(1, n_epochs+1):
    loss = train(model, criterion, optimizer, X, y)

    if epoch % 10 == 0:
        print('epoch: {}, loss: {:4f}'.format(epoch, loss))


y_predicted = (model(X) >= 0.5).float()

score = (y_predicted==y).float().mean()
print(score)



epoch: 10, loss: 62.741653
epoch: 20, loss: 62.741653
epoch: 30, loss: 62.741653
epoch: 40, loss: 62.741653
epoch: 50, loss: 62.741653
epoch: 60, loss: 62.741653
epoch: 70, loss: 62.741653
epoch: 80, loss: 62.741653
epoch: 90, loss: 62.741653
epoch: 100, loss: 62.741653
tensor(0.3726)


In [12]:
class LinearRegression(nn.Module):
    def __init__(self, num_features):
        super().__init__() # 부모의 생성자를 호출
        self.linear == nn.Linear(num_features, 1) #num_features 개의 특성을 입력받는 선형 모델 객체를 생성
        self.sigmoid = nn.Sigmoid()

    def forward(self, X):
        out = self.linear(X) # 생성자에서 만든 타깃으 추론하고 반환
        return out


model = LinearRegression(13)

AttributeError: 'LinearRegression' object has no attribute 'linear'

In [14]:
# 3.2.4 배치학습


#데이터를 조금씩 나눠서 처리하는 배치(batch)라는 개념 등장

from sklearn import datasets
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

dataset = datasets.load_breast_cancer()
X, y = dataset['data'], dataset['target']
X = torch.FloatTensor(X)
y = torch.FloatTensor(y).view(-1, 1)

x = (X - torch.mean(X)) / torch.std(X)

dset = TensorDataset(X, y)
loader = DataLoader(dset, batch_size=256, shuffle = True) 

class NeuralNetwork(nn.Module):

    def __init__(self, num_features):
        super().__init__() # 상속받아 생성한 객체, 부모의 생성자를 호출
        self.linear1 = nn.Linear(num_features, 4) # num_features개의 특성을 입력받는 은닉층 노드를 4개 생성
        self.relu = nn.ReLU() # 렐루 함수 객체 생성
        self.linear2 = nn.Linear(4, 1) #4개의 값을 입력받는 출력층 노드 1개를 생성
        self.sigmoid = nn.Sigmoid()

    def forward(self, X):
        out = self.linear1(X)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        return out

model = NeuralNetwork(30)
criterion = nn.BCELoss() # 이진 크로스 엔트로피 손실 함수 객체를 생성
optimizer = optim.SGD(model.paramters(), lr=0.1) # 확률적 경사 하강법 옵티마이저 객체를 생성

def train(model, criterion, optimizer, loader):

    epoch_loss = 0

    for X_batch, y_batch in loader:
        optimizer.zero_grad()
        hypothesis = model(X_batch)
        loss = criterion(hypothesis, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(loader)


n_epochs = 100
for epoch in range(1, n_epochs+1):
    loss = train(model, criterion, optimizer, loader)

    if epoch % 10 ==0:
        print(epoch, loss)






AttributeError: 'NeuralNetwork' object has no attribute 'paramters'

In [18]:
# 3.3 전결합 신경망

# 3.3.1 손글씨 이미지 분류하기
from torchvision import datasets
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

train_dataset = datasets.MNIST('./', train=True, download=True)
test_dataset = datasets.MNIST('./', train=False, download=True)

X_train, y_train = train_dataset.data / 255, train_dataset.targets
X_test, y_test = test_dataset.data / 255, test_dataset.targets 

X_train, X_test = X_train.view(-1, 784), X_test.view(-1, 784)

train_dset = TensorDataset(X_train, y_train)
test_dset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dset, batch_size=32, shuffle=True)
test_loader = DataLoader(train_dset, batch_size=32, shuffle=True)

class DNN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.hidden_layer1 = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU()
        )
        self.hidden_layer2 = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU()
        )
        self.output_layer = nn.Linear(128, 10)

def forward(self, X):
    out = self.hidden_layer1(X)
    out = self.hidden_layer2(out)
    out = self.output_layer(out)
    return out

device = 'cuda' if torch.cuda.is_available() else 'cpu' # 그래픽 카드 사용이 가능할 경우 그래픽 카드로 연산하도록 설정
model = DNN(784).to(device) # 784 개의 값을 입력받는 로지스틱 회귀 모델 객체를 생성
criterion = nn.CrossEntropyLoss()
optimizer  = optim.Adam(model.parameters(), lr=0.0001)

def train(model, criterion, optimizer, loader):
    epoch_loss = 0
    epoch_acc = 0

    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch(device)
        optimizer.zero_grad()
        hypothesis = model(X_batch)
        loss = criterion(hypothesis, y_batch)
        loss.backward()
        optimizer.step()
        y_predicted = torch.argmax(hypothesis, 1)
        acc = (y_predicted == y_batch).float().mean()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(loader), epoch_acc / len(loader)

def evaluate(model, criterion, loader):
    epoch_loss = 0
    epoch_acc = 0

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            hypothesis = model(X_batch)
            loss = criterion(hypothesis, y_batch)
            y_predicted = torch.argmax(hypothesis, 1)
            acc = (y_predicted == y_batch).foat().mean()
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(loader), epoch_acc /len(loader)


n_epochs = 20

for epoch in range(1, n_epochs+1):
    loss, acc = train(model, criterion, optimizer, train_loader)
    test_loss, test_acc = evaluate(model, criterion, test_loader)

    print(epoch, loss, acc, test_loss, test_acc)



TypeError: 'Tensor' object is not callable

In [22]:
# 합성곱 신경망
# 3.4.1 합성곱 신경망 개념

from torchvision import datasets
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim



train_dataset = dataset.MNIST('./', train=True, download=True)
test_dataset = dataset.MNIST('./', train=False, download=True)

X_train, y_train = train_dataset.data / 255, train_dataset.targets
X_test, y_test = test_dataset.data / 255, test_dataset.targets


AttributeError: MNIST